<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Overview</a></span></li></ul></li><li><span><a href="#Analyze-a-DD-SIMCA-Model" data-toc-modified-id="Analyze-a-DD-SIMCA-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analyze a DD-SIMCA Model</a></span><ul class="toc-item"><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load the Data</a></span></li><li><span><a href="#A-Simple-Model" data-toc-modified-id="A-Simple-Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>A Simple Model</a></span></li><li><span><a href="#An-Optimized-Pipeline" data-toc-modified-id="An-Optimized-Pipeline-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>An Optimized Pipeline</a></span></li><li><span><a href="#Create-an-Explainer" data-toc-modified-id="Create-an-Explainer-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Create an Explainer</a></span></li><li><span><a href="#A-Manual-Improvement" data-toc-modified-id="A-Manual-Improvement-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>A Manual Improvement</a></span></li></ul></li><li><span><a href="#Analyze-an-EllipticManifold-Model" data-toc-modified-id="Analyze-an-EllipticManifold-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analyze an EllipticManifold Model</a></span></li><li><span><a href="#Analyze-a-PLS-DA-Model" data-toc-modified-id="Analyze-a-PLS-DA-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analyze a PLS-DA Model</a></span><ul class="toc-item"><li><span><a href="#Examine-Categories" data-toc-modified-id="Examine-Categories-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Examine Categories</a></span></li><li><span><a href="#How-are-decisions-made?" data-toc-modified-id="How-are-decisions-made?-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>How are decisions made?</a></span></li></ul></li></ul></div>

In [ ]:
using_colab = 'google.colab' in str(get_ipython())
if using_colab:
    !git clone https://github.com/mahynski/chemometrics.git
    !cd chemometrics; pip3 install -r requirements.txt
else:
    import sys
    sys.path.append('../../')

import chemometrics

import matplotlib.pyplot as plt
%matplotlib inline

import watermark
%load_ext watermark

%load_ext autoreload
%autoreload 2

import shap
shap.initjs()

In [ ]:
import imblearn
import sklearn

from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd

In [ ]:
import bokeh
from bokeh.plotting import output_notebook
from chemometrics.utils import bokeh_color_spectrum
from chemometrics.utils import color_spectrum
output_notebook()

Overview
-------------
This notebook demonstrates the use of SHAP to explain model predictions.

For "soft" models / OCC, we need to look at the likelihood a sample is assigned as an inlier or not; when models combine many of these (e.g., PLS-DA) we need to pull those apart to inspect each known class.  Typically, a model may be trained and TEFF optimized.  From this, optimal hyperparameters can be extracted then individual models retrained and examined to explain predictions. 

In [ ]:
%watermark -t -m -v --iversions

# Analyze a DD-SIMCA Model

## Load the Data

In [ ]:
# Let's load some data from the tests/ for this example
if using_colab:
    loc = 'https://raw.githubusercontent.com/mahynski/chemometrics/master/tests/data/simca_train.csv'
else:
    loc = '../tests/data/simca_train.csv'
df = pd.read_csv(loc)

In [ ]:
# You can see that samples are rows, columns are different features
df.head()

In [ ]:
raw_x = np.array(df.values[:,3:], dtype=float) # Extract features
raw_y = np.array(df['Class'].values, dtype=str) # Take the class as the target

X_train = raw_x

In [ ]:
from chemometrics.classifier.simca import DDSIMCA_Model
from chemometrics.preprocessing.scaling import CorrectedScaler
from chemometrics.preprocessing.filter import SavGol

## A Simple Model

In [ ]:
# Here the data is spectra so we will not scale the X data
model = DDSIMCA_Model(n_components=7, alpha=0.05, gamma=0.01, scale_x=False)
_ = model.fit(X_train, raw_y)
_ = model.visualize(X_train, raw_y)

In [ ]:
mask1 = model.predict_proba(X_train)[:,0] > 0.5
mask2 = model.predict(X_train)
np.all(mask1 == mask2)

In [ ]:
model.predict(X_train)

In [ ]:
model.accuracy(X_train, np.array([True]*len(X_train)))

In [ ]:
model.score(X_train, np.array([True]*len(X_train))) # Negative log loss

## An Optimized Pipeline

In [ ]:
import imblearn

# Let's use some Pareto scaling just as an illustration, even though this may not be ideal with spectral data
pipeline = imblearn.pipeline.Pipeline(steps=[
    ("scaling", 
     CorrectedScaler(
        with_mean=True, 
        with_std=True, 
        pareto=True, 
        biased=False
     )
    ),
    ("savgol",
     SavGol(
         window_length=11, 
         deriv=2, 
         polyorder=3
     )
    ),
    ("simca", 
     DDSIMCA_Model(
         n_components=7, 
         alpha=0.05, 
         scale_x=False
     )
    )
])

# Hyperparameters of pipeline steps are given in standard notation: step__parameter_name
param_grid = [{
    'scaling__pareto': [True, False],
    'scaling__with_std': [True, False],
    'savgol__window_length': [5, 13],
    'savgol__deriv': [0, 1, 2],
    'savgol__polyorder': [3],
    'simca__n_components': np.arange(1, 10, 2),
}]

model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    n_jobs=-1,
    cv=sklearn.model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=0),
    error_score=0,
    refit=True,
)

_ = model.fit(
    X_train, 
    np.array([True]*len(X_train))
)

In [ ]:
model.best_params_ 

In [ ]:
# model.cv_results_ # View full CV results

In [ ]:
# Look for any ties - in this case, not substantially different
np.array(model.cv_results_['params'])[model.cv_results_['rank_test_score'] == 1]

In [ ]:
model.predict_proba(X_train)[:10]

## Create an Explainer

In [ ]:
# If you have a small dataset with only a few features you can use the entire training dataset for 
# the background (data). In problems with more features you probably want to pass only the median 
# of the training dataset, or weighted k-medians.
# See https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/model_agnostic/Iris%20classification%20with%20scikit-learn.html
# Since we have some spectral data with > 400 features, we can just use a random subsample of 10 as 
# the background.

# See SHAP documentation for a discussion on the utility and impact of using a "squashing function" to 
# go from an unbounded "margin space" (raw model output) to a bounded probability space.
# https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/model_agnostic/Squashing%20Effect.html#Probability-space-explaination
explainer = shap.KernelExplainer(
    model = model.predict_proba, # Use probability function to "squash"
    data = shap.sample(X_train, 10, random_state=0), # vs. data = X_train to use full training set
)

In [ ]:
# If there are many features to explain, this can fail to converge. You can choose to look at just
# the top features as shown below.  Feature correlation can also be a concern.  Using a smaller subset
# of decorrelated inputs.

# Consider BorutaShap, Heirarchical clustering, etc.

shap_values = explainer.shap_values(X_train, 
                                    nsamples='auto', # Can increase for lower variance
                                    l1_reg='num_features({})'.format(
                                        np.min(
                                            [X_train.shape[1], 20] # Only look at (up to) the top 20 features
                                        )
                                    )
                                   )

In [ ]:
explainer.expected_value 

In [ ]:
np.mean(model.predict_proba(X_train)[:,0]) 
# If the background sampling was bad, the mean model prediction will differ greatly from the 
# explainer.expected_value. If this is large, consider increasing the number of background samples.

In [ ]:
shap.force_plot(
    base_value=explainer.expected_value[0], # [0] to look at inlier probability
    shap_values=shap_values[0], # [0] to look at inlier probability
    features=X_train, # Feature values for visualization
    feature_names=['Channel {}'.format(i) for i in range(X_train.shape[1])] # Feature names for visualization
)

In [ ]:
# Look at the lowest probability example
look_at = np.argmin(model.predict_proba(X_train)[:,0]) 

shap.force_plot(
    base_value=explainer.expected_value[0], # [0] to look at inlier probability
    shap_values=shap_values[0][look_at,:], # [0] to look at inlier probability
    features=raw_x[look_at,:], # Feature values for visualization
    feature_names=['Channel {}'.format(i) for i in range(X_train.shape[1])] # Feature names for visualization
)

In [ ]:
shap.decision_plot(
    explainer.expected_value[0], 
    shap_values[0][look_at,:],
    feature_names=['Channel {}'.format(i) for i in range(raw_x.shape[1])]
)

In [ ]:
ax = color_spectrum(
    y=X_train[look_at,:], 
    x=np.arange(X_train.shape[1]),
    importance_values=shap_values[0][look_at], 
    figsize=(20,5),
    cmap='seismic',
    bounds=(-0.05, 0.05),
    background=True
)
ax.set_title('Worst Example')

# So the spectrum that is most strongly predicted to be an outlier, even though it is NOT is due mostly to
# parts of the spectrum that are at low wavenumbers are likely not relevant. This may not be a good 
# model. In part, this could be due to the fact that those values are poorly sampled/measured so in practice
# you might consider getting rid of that part of the spectrum before fitting. The model have learned something
# you didn't want it to!

In [ ]:
best = np.argmax(model.predict_proba(X_train)[:,0])
ax = color_spectrum(
    y=X_train[best,:], 
    x=np.arange(X_train.shape[1]),
    importance_values=shap_values[0][best], 
    figsize=(20,5),
    cmap='seismic',
    bounds=(-0.05, 0.05),
    background=True
)
ax.set_title('Best Example')

# The features that are signalling this spectrum is part of the training class look like they are from 
# reasonable parts of the spectrum.

In [ ]:
shap.summary_plot(
    shap_values=shap_values[0],
    features=X_train,
    plot_type='bar'
)

## A Manual Improvement

In [ ]:
# Let's just choose to trim off the initial part of the spectrum
X_refined = X_train[:, 50:]

In [ ]:
import imblearn

pipeline = imblearn.pipeline.Pipeline(steps=[
    ("scaling", 
     CorrectedScaler(
        with_mean=True, 
        with_std=True, 
        pareto=True, 
        biased=False
     )
    ),
    ("savgol",
     SavGol(
         window_length=11, 
         deriv=2, 
         polyorder=3
     )
    ),
    ("simca", 
     DDSIMCA_Model(
         n_components=7, 
         alpha=0.05, 
         scale_x=False
     )
    )
])

# Hyperparameters of pipeline steps are given in standard notation: step__parameter_name
param_grid = [{
    'scaling__pareto': [True, False],
    'scaling__with_std': [True, False],
    'savgol__window_length': [5, 13],
    'savgol__deriv': [0, 1, 2],
    'savgol__polyorder': [3],
    'simca__n_components': np.arange(1, 10, 2),
}]

model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    n_jobs=-1,
    cv=sklearn.model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=0),
    error_score=0,
    refit=True,
)

_ = model.fit(
    X_refined, 
    np.array([True]*len(X_refined))
)

In [ ]:
# Look for any ties - in this case, not substantially different
np.array(model.cv_results_['params'])[model.cv_results_['rank_test_score'] == 1]

In [ ]:
model.best_params_

In [ ]:
explainer = shap.KernelExplainer(
    model = model.predict_proba, # Use probability function to "squash"
    data = shap.sample(X_refined, 10, random_state=0), 
)

In [ ]:
explainer.expected_value

In [ ]:
np.mean(model.predict_proba(X_refined)[:,0]) 

In [ ]:
shap_values = explainer.shap_values(X_refined, 
                                    nsamples='auto', # Can increase for lower variance
                                    l1_reg='num_features({})'.format(
                                        np.min(
                                            [X_refined.shape[1], 20] # Only look at (up to) the top 20 features
                                        )
                                    )
                                   )

In [ ]:
worst = np.argmin(model.predict_proba(X_refined)[:,0])

ax = color_spectrum(
    x=np.arange(X_refined.shape[1]),
    y=X_refined[worst,:], 
    importance_values=shap_values[0][worst], 
    figsize=(20,5),
    cmap='seismic',
    bounds=(-0.05, 0.05),
    background=True
)
ax.set_title('Worst Example')

In [ ]:
best = np.argmax(model.predict_proba(X_refined)[:,0])

ax = color_spectrum(
    x=np.arange(X_refined.shape[1]),
    y=X_refined[best,:], 
    importance_values=shap_values[0][best], 
    figsize=(20,5),
    cmap='seismic',
    bounds=(-0.05, 0.05),
    background=True
)
ax.set_title('Best Example')

In [ ]:
# You have to decide if this makes sense or not.
shap.summary_plot(
    shap_values=shap_values[0],
    features=X_refined,
    plot_type='bar'
)

In [ ]:
# We can color an individual spectrum based on the features important to its own classification.
# This is useful to diagnose and individual prediction.
chosen = 0
bokeh_color_spectrum(x=np.arange(X_refined.shape[1]), 
                     y=X_refined[chosen, :], 
                     importance_values=shap_values[0][chosen])

In [ ]:
# Or, we can color by the average SHAP importance.
# This is useful for visualizing how the model makes predictions on average.
chosen = 0
bokeh_color_spectrum(x=np.arange(X_refined.shape[1]), 
                     y=X_refined[chosen, :], 
                     importance_values=np.mean(np.abs(shap_values[0]), axis=0) # Mean ABSOLUTE value
                    )

# Analyze an EllipticManifold Model

In [ ]:
import sklearn.manifold
from chemometrics.manifold.elliptic import EllipticManifold

In [ ]:
mani = sklearn.manifold.Isomap
kwargs = {"n_neighbors":10, 
          "n_components":2, # Choose 2 for visualization
          "metric":'minkowski',
          "p":2, 
         }
model = EllipticManifold(0.05, mani, kwargs)

In [ ]:
_ = model.fit(X_train, raw_y)

In [ ]:
model.visualize([X_train], ["Training Set"])

In [ ]:
explainer = shap.KernelExplainer(
    model = model.predict_proba, # Use probability function to "squash"
    data = shap.sample(X_train, 10, random_state=0), 
)

In [ ]:
shap_values = explainer.shap_values(X_train, 
                                    nsamples='auto', # Can increase for lower variance
                                    l1_reg='num_features({})'.format(
                                        np.min(
                                            [X_train.shape[1], 20] # Only look at (up to) the top 20 features
                                        )
                                    )
                                   )

In [ ]:
explainer.expected_value

In [ ]:
np.mean(model.predict_proba(X_train)[:,0]) 

In [ ]:
# On AVERAGE, it looks like the most important parts of the spectrum are somewhere around channel ~80
shap.summary_plot(
    shap_values=shap_values[0],
    features=X_train,
    plot_type='bar'
)

In [ ]:
# Or, we can color by the average SHAP importance.
# This is useful for visualizing how the model makes predictions on average.
chosen = 0
bokeh_color_spectrum(x=np.arange(X_train.shape[1]), 
                     y=X_train[chosen, :], 
                     importance_values=np.mean(np.abs(shap_values[0]), axis=0) # Mean ABSOLUTE value
                    )

In [ ]:
# This model seems to focus on characterizing the spectra based on a region of high variance.
chosen = 0
bokeh_color_spectrum(x=np.arange(X_train.shape[1]), 
                     y=X_train[chosen, :], 
                     importance_values=np.std(X_train, axis=0)**2 # Variance in the training data
                    )

# Analyze a PLS-DA Model

In [ ]:
# Let's load some data from the tests/ for this example
if using_colab:
    loc = 'https://raw.githubusercontent.com/mahynski/chemometrics/master/tests/data/plsda3_train.csv'
else:
    loc = '../tests/data/plsda3_train.csv'
df = pd.read_csv(loc)

In [ ]:
# now using SITE data
df.head()

In [ ]:
element_names = df.columns[3:]
raw_x = np.array(df.values[:,3:], dtype=float) # Extract features
raw_y = np.array(df['Class'].values, dtype=str) # Take the class as the target

In [ ]:
from chemometrics.classifier.plsda import PLSDA

In [ ]:
# Here the data are elemental levels so we will scale the X data
plsda = PLSDA(n_components=5, 
              alpha=0.05, 
              gamma=0.01, 
              not_assigned='UNKNOWN', 
              style="soft", 
              scale_x=True)

In [ ]:
_ = plsda.fit(raw_x, raw_y)

In [ ]:
_ = plsda.visualize(styles=['hard', 'soft'])

In [ ]:
plsda.predict(raw_x)

In [ ]:
plsda.categories

In [ ]:
plsda.predict_proba(raw_x)[:10]

In [ ]:
# plsda.predict_proba predicts the probability for EACH individual class (in columns).
# Athough these do not sum to unity, we can still analyze each column individually with SHAP!

explainer = shap.KernelExplainer(
    model = plsda.predict_proba, 
    data = shap.sample(raw_x, 10, random_state=0), # vs. data = raw_x to use full training set
)

In [ ]:
shap_values = explainer.shap_values(raw_x, 
                                    nsamples='auto', # Can increase for lower variance
                                    l1_reg='num_features({})'.format(
                                        np.min(
                                            [raw_x.shape[1], 20] # Only look at (up to) the top 20 features
                                        )
                                    )
                                   )

In [ ]:
len(shap_values) # There is a set of shap values for each class

In [ ]:
explainer.expected_value # Average probability of being an inlier for each class in the training set

## Examine Categories

In [ ]:
chosen_class = 2 # Which category? 0, 1, or 2

In [ ]:
sv = shap_values[chosen_class]
plsda.categories[chosen_class]

shap.summary_plot(
    shap_values=sv,
    features=raw_x,
    feature_names=element_names,
    plot_type='bar'
)

In [ ]:
# You can see that the SHAP values add up to the probability a point is an inlier in a class:

examine_pt = 12

shap_sum = np.sum(shap_values[chosen_class][examine_pt]) + explainer.expected_value[chosen_class]
pred_prob = plsda.predict_proba(raw_x)[examine_pt][chosen_class]

print(shap_sum, pred_prob, shap_sum-pred_prob)

In [ ]:
shap.summary_plot(
    shap_values=sv,
    features=raw_x,
    feature_names=element_names,
    plot_type='violin'
)

In [ ]:
shap.force_plot(
    base_value=explainer.expected_value[chosen_class],
    shap_values=sv, 
    features=raw_x,
    feature_names=element_names
)

In [ ]:
# Look at the lowest probability example
look_at = np.argmin(plsda.predict_proba(raw_x)[:,chosen_class]) 

shap.force_plot(
    base_value=explainer.expected_value[chosen_class], 
    shap_values=sv[look_at,:], 
    features=raw_x[look_at,:], 
    feature_names=element_names
)

In [ ]:
shap.decision_plot(
    explainer.expected_value[chosen_class], 
    sv[look_at,:],
    feature_names=element_names.tolist()
)

In [ ]:
# Look at the highest probability example
look_at = np.argmax(plsda.predict_proba(raw_x)[:,chosen_class]) 

shap.force_plot(
    base_value=explainer.expected_value[chosen_class], 
    shap_values=sv[look_at,:], 
    features=raw_x[look_at,:], 
    feature_names=element_names
)

In [ ]:
shap.decision_plot(
    explainer.expected_value[chosen_class], 
    sv[look_at,:],
    feature_names=element_names.tolist()
)

## How are decisions made?

In [ ]:
# Let's assess the specificity of the model - we can look at examples of when a sample is predicted to belong
# to a class that it does not.

In [ ]:
chosen_class = 0
sv = shap_values[chosen_class]
print('Examining model for : {}'.format(plsda.categories[chosen_class])) # Let's look at samples which belong to this category

In [ ]:
compare_class = (chosen_class+1)%3 # Let's see how the model predicts for other classes
print('These samples belong to : {}'.format(plsda.categories[compare_class]))

# Look at samples that belong to compare_class
mask = raw_y == plsda.categories[compare_class]

# These should NOT be predicted to belong to chosen_class
wrong = plsda.predict_proba(raw_x[mask])[:,chosen_class] > 0.5

shap.decision_plot(
    explainer.expected_value[chosen_class], 
    sv[mask,:], 
    feature_names=element_names.tolist(),
    highlight=wrong
)

In [ ]:
# Although none are wrong here, some are around 40% and the threshold for class membership is 50%, so 
# those might be of interest for further investigation.

In [ ]:
compare_class = (chosen_class+2)%3 # Let's see how the model predicts for other classes
print('These samples belong to : {}'.format(plsda.categories[compare_class]))

# Look at samples that belong to compare_class
mask = raw_y == plsda.categories[compare_class]

# These should NOT be predicted to belong to chosen_class
wrong = plsda.predict_proba(raw_x[mask])[:,chosen_class] > 0.5

shap.decision_plot(
    explainer.expected_value[chosen_class], 
    sv[mask,:], 
    feature_names=element_names.tolist(),
    highlight=wrong
)

In [ ]:
# These ones that are wrong seem like outliers, whereas most are predicted correctly, confidently

In [ ]:
# However, in the latent space, these wrong ones appear to be just "on the edge" and could be included
# if alpha (type 1 error rate) was adjusted.  That is not the point - we WANT a few of those on the edge
# to be excluded to draw the boundary, so this appears to be reasonable.
t = plsda.transform(raw_x[mask][~wrong])
plt.plot(t[:,0], t[:,1], 'o')

t = plsda.transform(raw_x[mask][wrong])
plt.plot(t[:,0], t[:,1], 'o')

In [ ]:
# First incorrect point
shap.force_plot(
    base_value=explainer.expected_value[chosen_class], 
    shap_values=sv[mask][wrong,:][0], 
    features=raw_x[mask][wrong,:][0], 
    feature_names=element_names
)

In [ ]:
# Second incorrect point
shap.force_plot(
    base_value=explainer.expected_value[chosen_class], 
    shap_values=sv[mask][wrong,:][1], 
    features=raw_x[mask][wrong,:][1], 
    feature_names=element_names
)

In [ ]:
compare_class = chosen_class # Let's see how the model predicts for other classes
print('These samples belong to : {}'.format(plsda.categories[compare_class]))

# Look at samples that belong to compare_class
mask = raw_y == plsda.categories[compare_class]

# These SHOULD be predicted to belong to chosen_class
wrong = plsda.predict_proba(raw_x[mask])[:,chosen_class] < 0.5

shap.decision_plot(
    explainer.expected_value[chosen_class], 
    sv[mask,:], 
    feature_names=element_names.tolist(),
    highlight=wrong
)

In [ ]:
# Although a few are wrong, they are pretty close to the "pack"